In [1]:
import sys
import logging
import requests
import base64
import json
import pymysql
import csv
import boto3
from datetime import datetime
import pandas as pd
import jsonpath

### Connecting to Amazon RDS

In [2]:
## mysql -h testdb-spotify.c1ljsqo7birx.us-west-2.rds.amazonaws.com -P 3306 -D production -u admin -p
host="testdb-spotify.c1ljsqo7birx.us-west-2.rds.amazonaws.com"
port=3306
username="admin"
password="Wlqtnsdl11061310*"
database="production"

connection=pymysql.connect(host=host, user=username, password=password, database=database, port=port, use_unicode=True, charset="utf8")
cursor=connection.cursor()  

### Funtion to add rows in a table

In [3]:
# query = """
#INSERT INTO artists (id, name, follwers, popularity, url, image_url)
#VALUES ('{}', '{}', '{}', '{}', '{}', '{}')
#ON DUPILICATE KEY UPDATE id='{}', name='{}', follwers='{}', popularity='{}', url='{}', image_url='{}'
#""".format(artist['id'], artist['name'], artist['followers'], artist['popularity'], artist['url'], artist['image_url'],
#           artist['id'], artist['name'], artist['followers'], artist['popularity'], artist['url'], artist['image_url'])


def insert_row(cursor, data, table_name):
    
    column_name = ', '.join(data.keys())
    print('columns:', column_name)
    
    placeholders = ', '.join(['%s'] * len(data)) #len(data) = the number of columns
    #%s, %s, %s, %s, %s, %s

    key_placeholders = ', '.join(['{}=%s'.format(k) for k in data.keys()])
    #id=%s, name=%s, followers=%s, popularity=%s, url=%s, image_url=%s
    
    query = "INSERT INTO %s (%s) VALUES (%s) ON DUPLICATE KEY UPDATE %s" % (table_name, column_name, placeholders, key_placeholders)
    cursor.execute(query, list(data.values())*2)

### Access to Spotify API

In [4]:
client_id="b03942aa01eb477cacb26e9bf8d3da58"
response_type="code"
redirect_uri="http://localhost:8888"
#scope="user-read-private%20user-read-email"
endpoint1="https://accounts.spotify.com/authorize?client_id={}&response_type={}&redirect_uri={}".format(client_id, response_type, redirect_uri)
#endpoint1="https://accounts.spotify.com/authorize?client_id={}&response_type={}&redirect_uri={}&scope={}".format(client_id, response_type, redirect_uri, scope)
res1=requests.get(endpoint1)

print(res1.status_code)
print(res1.text)
print(endpoint1)

200

<!DOCTYPE html>
<html id="app" lang="en" dir="ltr" ng-csp ng-strict-di>
<head>
  <meta charset="utf-8">
  <title ng-bind="(&#39;loginTitle&#39; | localize) + &#39; - Spotify&#39;">Spotify</title>
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
  <base href="/">
  <link rel="icon" href="https://accounts.scdn.co/sso/images/favicon.ace4d8543bbb017893402a1e9d1ac1fa.ico">
  <link href="https://accounts.scdn.co/sso/css/index.3ff124dfd72bdb0a5a3d.css" media="screen" rel="stylesheet">

  <script defer src="https://accounts.scdn.co/sso/js/index.3ff124dfd72bdb0a5a3d.js" sp-bootstrap></script>
  <meta ng-non-bindable sp-bootstrap-data='{"phoneFeatureEnabled":false,"previewEnabled":false,"user":false,"tpaState":"AQCbRWo9g6O/pSIwVdQdkSm9aECbAf5eJNhp/ewAahY26XTGdfxJFdxGm7tx371E/nO8rDHvE8Gr+VTxNig+3vs85FJEiNxCDy4Jh64w7EvkjC0zd5v44cHtY0YuAwHUvfd5busiGEn/kkKVVFB9Bsr8QkmtcsN6ZsKIjpMsJK5vpcP9pgHQRAfJ7Yw3Mb57VskggudXLlWpXmCFgP6hY5ZF0/8JCUw

In [5]:
data={
    "grant_type":"authorization_code",
    "code":"AQDEo4kcMCU2cgS3-tX1LsI-9koCdvW7PJ3UYfsUz9gnAPg-eVKwdrYlCEGc9GHJBhidvIsRuhFsfHPwe458iqbd2LuZNtQLXxrSr_3SO_RPN8j0OMLBVxugLofZHXIOZs_NpiisqlO0Cduiu7RvcfT-fWFt9kGuKA",
    "redirect_uri":"http://localhost:8888"
}


client_id="b03942aa01eb477cacb26e9bf8d3da58"
client_secret="fd61807827344135a98ec16305e34bb2"
encoded=base64.b64encode('{}:{}'.format(client_id, client_secret).encode('utf-8')).decode('ascii')
headers={"Authorization":"Basic {}".format(encoded)}

endpoint2="https://accounts.spotify.com/api/token"

res2=requests.post(endpoint2, data=data, headers=headers)

print(res2.status_code)
print(res2.text) #type=string

200
{"access_token":"BQA9xEco3CPP5btITGGRervheNV_4J0vN_CB_WDq6B6ot84KtvCVHXXH45j6-EYTfxT8xu1MVj9StGnjKdb11czSMZkz5Nj4UHzxoAgIOMjdtelxcP18KpGeFWLxCzqGNOpp-EvlQsvaxMp88XuoBUA5jkQOxyit31eRHg","token_type":"Bearer","expires_in":3600,"refresh_token":"AQCxoLq4q6MH84-uVQJzkejn0S4UyHSkB6yxDV1QRcFQgHXNQ2pF3gMIbWsD1YvbyG4Ux8uCK2HRrJCAoxMDiQxcbshnpoQ1IBcN3VeN-u0Y6FtxBTRPRD7GAKl_0gjrYMs"}


In [7]:
print(res2.json())
json.loads(res2.text) #type=dict

access_token=json.loads(res2.text)['access_token']
headers2={"Authorization":"Bearer {}".format(access_token)}

{'access_token': 'BQA9xEco3CPP5btITGGRervheNV_4J0vN_CB_WDq6B6ot84KtvCVHXXH45j6-EYTfxT8xu1MVj9StGnjKdb11czSMZkz5Nj4UHzxoAgIOMjdtelxcP18KpGeFWLxCzqGNOpp-EvlQsvaxMp88XuoBUA5jkQOxyit31eRHg', 'token_type': 'Bearer', 'expires_in': 3600, 'refresh_token': 'AQCxoLq4q6MH84-uVQJzkejn0S4UyHSkB6yxDV1QRcFQgHXNQ2pF3gMIbWsD1YvbyG4Ux8uCK2HRrJCAoxMDiQxcbshnpoQ1IBcN3VeN-u0Y6FtxBTRPRD7GAKl_0gjrYMs'}


### Creating file 'top-tracks' (.json or .parquet)

In [8]:
cursor.execute("SELECT id FROM artists limit 10")
top_track_keys={'id': 'id', 'name':'name', 'poularity':'popularity', 'external_url':'external_urls.spotify'}
top_tracks = []

for (id, ) in cursor.fetchall():
    URL = 'https://api.spotify.com/v1/artists/{}/top-tracks'.format(id)
    params = {'country':'US'}
    r = requests.get(URL, params=params, headers=headers2)
    raw = json.loads(r.text)
    
    for i in raw['tracks']:
        top_track = {}
        for k, v in top_track_keys.items():
            top_track.update({k: jsonpath.jsonpath(i, v)})
            top_track.update({'artist_id': id})
            top_tracks.append(top_track)

track_ids = [track['id'][0] for track in top_tracks]

top_tracks = pd.DataFrame(top_tracks)
top_tracks.to_parquet('top-tracks.parquet', engine='pyarrow', compression='snappy')


# with open('top_tracks.json', 'w') as f:
#     for track in top_tracks:
#         json.dump(track, f)
#         f.write(os.linesp)
        
        

dt = datetime.utcnow().strftime("%Y-%m-%d")

### Storing 'top-tracks.parquet' in S3

In [9]:
s3 = boto3.resource('s3')
object = s3.Object('top-tracks-audio-feature', 'top-tracks/dt={}/top-tracks.parquet'.format(dt))
data = open('top-tracks.parquet', 'rb')
object.put(Body=data)

{'ResponseMetadata': {'RequestId': 'EHSBGQZN1Y6N6JH6',
  'HostId': 'AoX2AVBNGiDRfHoMmx2ySaSJ1fg4OgtFKueqmt73GNzKgFbOF2FJP8M2to6O5o7Cyx5dm1AWXFg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'AoX2AVBNGiDRfHoMmx2ySaSJ1fg4OgtFKueqmt73GNzKgFbOF2FJP8M2to6O5o7Cyx5dm1AWXFg=',
   'x-amz-request-id': 'EHSBGQZN1Y6N6JH6',
   'date': 'Sat, 23 Oct 2021 23:42:10 GMT',
   'etag': '"d51187d8acb836fba0994b6271fd2217"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d51187d8acb836fba0994b6271fd2217"'}

### Creating file 'audio-features.parquet'

In [10]:
print(track_ids[:10])
tracks_batch = [track_ids[i:i+100] for i in range(0, len(track_ids), 100)]
#print(tracks_batch[0])

audio_features = []
for batch in tracks_batch:
    
    ids=','.join(batch)
    URL = 'https://api.spotify.com/v1/audio-features/?ids={}'.format(ids)
    r = requests.get(URL, headers=headers2)
    raw = json.loads(r.text)
    audio_features.extend(raw['audio_features'])
    
audio_features = pd.DataFrame(audio_features)
audio_features.to_parquet('autio_features.parquet', engine='pyarrow', compression='snappy')

['2nMeu6UenVvwUktBCpLMK9', '2nMeu6UenVvwUktBCpLMK9', '2nMeu6UenVvwUktBCpLMK9', '2nMeu6UenVvwUktBCpLMK9', '1NZs6n6hl8UuMaX0UC0YTz', '1NZs6n6hl8UuMaX0UC0YTz', '1NZs6n6hl8UuMaX0UC0YTz', '1NZs6n6hl8UuMaX0UC0YTz', '6PUIzlqotEmPuBfjbwYWOB', '6PUIzlqotEmPuBfjbwYWOB']


### Storing 'audio-features.parquet' in S3

In [11]:
s3 = boto3.resource('s3')
object = s3.Object('top-tracks-audio-feature', 'audio-features/dt={}/top-tracks.parquet'.format(dt))
data = open('autio_features.parquet', 'rb')
object.put(Body=data)

{'ResponseMetadata': {'RequestId': 'W1QBGWNGDABYW6SP',
  'HostId': '7ViNz5/EHbvXQdRjlX5G9ZFvdk1+OtCVQZWGNBMofpf7Ed6F7KszknNM/58v0gSBTsFouYfITjU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '7ViNz5/EHbvXQdRjlX5G9ZFvdk1+OtCVQZWGNBMofpf7Ed6F7KszknNM/58v0gSBTsFouYfITjU=',
   'x-amz-request-id': 'W1QBGWNGDABYW6SP',
   'date': 'Sat, 23 Oct 2021 23:42:32 GMT',
   'etag': '"148965c31f0038285fa9ca936270e0a5"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"148965c31f0038285fa9ca936270e0a5"'}

In [14]:
print(top_tracks)

                           id               artist_id  \
0    [2nMeu6UenVvwUktBCpLMK9]  00FQb4jTyendYWaN8pK0wa   
1    [2nMeu6UenVvwUktBCpLMK9]  00FQb4jTyendYWaN8pK0wa   
2    [2nMeu6UenVvwUktBCpLMK9]  00FQb4jTyendYWaN8pK0wa   
3    [2nMeu6UenVvwUktBCpLMK9]  00FQb4jTyendYWaN8pK0wa   
4    [1NZs6n6hl8UuMaX0UC0YTz]  00FQb4jTyendYWaN8pK0wa   
..                        ...                     ...   
395  [53yTYusPQJ1AApL1hi0Dnc]  0BvkDsjIUla7X0k6CSWh1I   
396  [1YhFtqwcN138S6ng3MT1nN]  0BvkDsjIUla7X0k6CSWh1I   
397  [1YhFtqwcN138S6ng3MT1nN]  0BvkDsjIUla7X0k6CSWh1I   
398  [1YhFtqwcN138S6ng3MT1nN]  0BvkDsjIUla7X0k6CSWh1I   
399  [1YhFtqwcN138S6ng3MT1nN]  0BvkDsjIUla7X0k6CSWh1I   

                           name poularity  \
0         [Young And Beautiful]      [78]   
1         [Young And Beautiful]      [78]   
2         [Young And Beautiful]      [78]   
3         [Young And Beautiful]      [78]   
4               [Brooklyn Baby]      [77]   
..                          ...       ...   


In [13]:
print(audio_features)

     danceability  energy  key  loudness  mode  speechiness  acousticness  \
0           0.324   0.416   11    -8.920     0       0.0368         0.262   
1           0.324   0.416   11    -8.920     0       0.0368         0.262   
2           0.324   0.416   11    -8.920     0       0.0368         0.262   
3           0.324   0.416   11    -8.920     0       0.0368         0.262   
4           0.404   0.664    1    -6.698     1       0.0407         0.542   
..            ...     ...  ...       ...   ...          ...           ...   
395         0.471   0.897    9    -2.270     1       0.0597         0.193   
396         0.622   0.932   11    -4.660     0       0.0498         0.105   
397         0.622   0.932   11    -4.660     0       0.0498         0.105   
398         0.622   0.932   11    -4.660     0       0.0498         0.105   
399         0.622   0.932   11    -4.660     0       0.0498         0.105   

     instrumentalness  liveness  valence    tempo            type  \
0     